In [1]:
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as shc

In [2]:
df = pd.read_parquet('./in_2/train_base.parquet')

In [4]:
df.set_index("case_id")

,date_decision,MONTH,WEEK_NUM,target
case_id,,,,
0,2019-01-03,201901,0,0
1,2019-01-03,201901,0,0
2,2019-01-04,201901,0,0
3,2019-01-03,201901,0,0
4,2019-01-04,201901,0,1
...,...,...,...,...
2703450,2020-10-05,202010,91,0
2703451,2020-10-05,202010,91,0
2703452,2020-10-05,202010,91,0


In [6]:
df.count

<bound method DataFrame.count of          case_id date_decision   MONTH  WEEK_NUM  target
0              0    2019-01-03  201901         0       0
1              1    2019-01-03  201901         0       0
2              2    2019-01-04  201901         0       0
3              3    2019-01-03  201901         0       0
4              4    2019-01-04  201901         0       1
...          ...           ...     ...       ...     ...
1526654  2703450    2020-10-05  202010        91       0
1526655  2703451    2020-10-05  202010        91       0
1526656  2703452    2020-10-05  202010        91       0
1526657  2703453    2020-10-05  202010        91       0
1526658  2703454    2020-10-05  202010        91       0

[1526659 rows x 5 columns]>

In [12]:
df_1 = pd.read_parquet('./in_2/train_static_0_0.parquet').set_index("case_id")

In [13]:
df_2 = pd.read_parquet('./in_2/train_static_0_1.parquet').set_index("case_id")

In [14]:
df_3 = pd.read_parquet('./in_2/train_static_cb_0.parquet').set_index("case_id")

In [31]:
df_final = df.join(df_1).join(df_3)

In [54]:
df_t= df_final.loc[:,['interestrate_311L','isdebitcard_729L']]

In [126]:
df_t['both_null'] = np.where(df_t['interestrate_311L'].isnull() & df_t['isdebitcard_729L'].isnull(), 1, 0)

In [139]:
x = df_t[(df_t['interestrate_311L'].isnull()) & (df_t['isdebitcard_729L'].isnull())]

In [141]:
len(x)/len(df_final)

0.5304537555537943

In [142]:
y = df_t[(df_t['interestrate_311L'].isnull()) & (df_t['isdebitcard_729L'].notnull())]

In [144]:
len(y)/len(df_final)

0.05760421940983546

In [145]:
z = df_t[(df_t['interestrate_311L'].notnull()) & (df_t['isdebitcard_729L'].isnull())]

In [147]:
len(z)/len(df_final)

0.41178612905698

In [149]:
(len(x) + len(y) + len(x))/len(df_final)

1.118511730517424

In [122]:
df_t['test'] = np.where(((df_t['isdebitcard_729L'].isnull()) | (df_t['interestrate_311L'] == 0.45)) ,1, 0)

In [123]:
df_t

,interestrate_311L,isdebitcard_729L,both_null,test
0,0.4500,None,0,1
1,0.2999,None,0,1
2,0.4500,None,0,1
3,0.4200,None,0,1
4,0.4500,None,0,1
...,...,...,...,...
1526654,0.3900,None,0,1
1526655,0.0000,None,0,1
1526656,0.4500,None,0,1
1526657,NaN,False,0,0


In [85]:
df_t.dtypes

interestrate_311L    float64
isdebitcard_729L      object
both_null              int32
dtype: object

In [69]:
df_t.both_null.sum()

238

In [58]:
df_t.groupby(['interestrate_311L','isdebitcard_729L']).size()

interestrate_311L  isdebitcard_729L
0.000              False               215
0.181              False                21
0.433              False                 2
dtype: int64

In [50]:
df_t.mode()

,interestrate_311L,isdebitcard_729L
0,0.0,False


In [39]:
df_final['interestrate_311L']

0          0.4500
1          0.2999
2          0.4500
3          0.4200
4          0.4500
            ...  
1526654    0.3900
1526655    0.0000
1526656    0.4500
1526657       NaN
1526658    0.4000
Name: interestrate_311L, Length: 1526659, dtype: float64

,date_decision,MONTH,WEEK_NUM,target,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,...,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
case_id,,,,,,,,,,,,,,,,,,,,,
0,2019-01-03,201901,0,0,NaN,NaN,1917.6,0.0000,0.0,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,None,None
1,2019-01-03,201901,0,0,NaN,NaN,3134.0,0.0000,0.0,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,None,None
2,2019-01-04,201901,0,0,NaN,NaN,4937.0,0.0000,0.0,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,AL,None
3,2019-01-03,201901,0,0,NaN,NaN,4643.6,0.0000,0.0,1.0,...,1.0,1.0,NaN,NaN,0.000,0.00,NaN,BO,AL,None
4,2019-01-04,201901,0,1,NaN,NaN,3390.2,0.0000,0.0,1.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,AL,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2651088,2019-12-16,201912,49,0,0.0,117624.79,3045.0,4488.0000,0.0,0.0,...,0.0,0.0,44848.000,44848.000,44848.000,199664.67,8976.000,FO,None,2019-11-12
2651089,2019-12-16,201912,49,0,0.0,134887.40,1200.0,4382.8003,0.0,0.0,...,0.0,10.0,39392.000,39392.000,39392.000,209840.00,8765.601,FO,None,2019-11-12
2651090,2019-12-16,201912,49,0,0.0,69186.62,6000.0,0.0000,0.0,0.0,...,0.0,0.0,0.000,0.000,0.000,76455.62,NaN,FO,None,2019-11-12


,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,...,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
case_id,,,,,,,,,,,,,,,,,,,,,
40626,NaN,NaN,1976.2001,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,FO,AL,None
40704,NaN,NaN,3731.2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,FO,AL,None
40734,NaN,NaN,3731.2000,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,NaN,NaN,0.0,0.00,NaN,FO,AL,None
40737,0.0,NaN,2362.0000,0.0,0.0,0.0,2.0,0.0,0.0,1.0,...,0.0,1.0,NaN,NaN,0.0,0.00,NaN,FO,AL,None
40766,NaN,NaN,1143.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,FO,AL,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703450,0.0,176561.36,3675.4001,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,8.0,0.0,0.0,0.0,428159.66,14346.319,FO,None,None
2703451,0.0,301276.47,7088.6000,6191.6,0.0,0.0,5.0,0.0,0.0,5.0,...,0.0,3.0,68098.4,68098.4,68098.4,701247.30,40499.805,FO,None,None
2703452,0.0,14232.40,7788.8003,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,1.0,0.0,0.0,0.0,24002.00,NaN,BO,None,None


(0, 338)

,case_id,date_decision,MONTH,WEEK_NUM,target,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,...,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
0,0,2019-01-03,201901,0,0,NaN,NaN,1917.6,0.0000,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,None,None
1,1,2019-01-03,201901,0,0,NaN,NaN,3134.0,0.0000,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,None,None
2,2,2019-01-04,201901,0,0,NaN,NaN,4937.0,0.0000,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,AL,None
3,3,2019-01-03,201901,0,0,NaN,NaN,4643.6,0.0000,0.0,...,1.0,1.0,NaN,NaN,0.000,0.00,NaN,BO,AL,None
4,4,2019-01-04,201901,0,1,NaN,NaN,3390.2,0.0000,0.0,...,0.0,0.0,NaN,NaN,0.000,0.00,NaN,BO,AL,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003752,2651088,2019-12-16,201912,49,0,0.0,117624.79,3045.0,4488.0000,0.0,...,0.0,0.0,44848.000,44848.000,44848.000,199664.67,8976.000,FO,None,2019-11-12
1003753,2651089,2019-12-16,201912,49,0,0.0,134887.40,1200.0,4382.8003,0.0,...,0.0,10.0,39392.000,39392.000,39392.000,209840.00,8765.601,FO,None,2019-11-12
1003754,2651090,2019-12-16,201912,49,0,0.0,69186.62,6000.0,0.0000,0.0,...,0.0,0.0,0.000,0.000,0.000,76455.62,NaN,FO,None,2019-11-12
1003755,2651091,2019-12-16,201912,49,0,0.0,117331.00,11565.4,13781.2000,0.0,...,0.0,2.0,55084.402,55084.402,55084.402,168248.31,13781.200,FO,None,2019-11-30
